In [11]:
import pandas as pd

# Load data from CSV file
df = pd.read_csv('./Dal/DAL.csv')

# Assuming there's a date column in your CSV. If not, adjust accordingly.
# Make sure the date column is in datetime format
df['Date'] = pd.to_datetime(df['Date'])

In [12]:
df

,Date,Open,High,Low,Close,Adj Close,Volume
0,2007-05-03,21.750000,21.950001,20.549999,20.719999,18.365900,8052800
1,2007-05-04,21.250000,21.350000,20.700001,20.980000,18.596355,5437300
2,2007-05-07,21.200001,21.290001,20.200001,20.360001,18.046801,2646300
3,2007-05-08,20.000000,20.000000,19.299999,19.629999,17.399738,4166100
4,2007-05-09,19.750000,20.200001,19.629999,19.790001,17.541559,7541100
...,...,...,...,...,...,...,...
4119,2023-09-13,39.500000,40.099998,39.279999,39.549999,39.549999,11263000
4120,2023-09-14,40.200001,40.279999,38.930000,39.330002,39.330002,13190400
4121,2023-09-15,39.299999,40.110001,39.110001,39.900002,39.900002,9841100
4122,2023-09-18,39.580002,39.830002,38.730000,38.889999,38.889999,8517700


In [69]:
df.isna().count()

Date         4124
Open         4124
High         4124
Low          4124
Close        4124
Adj Close    4124
Volume       4124
dtype: int64

In [13]:
# Split data before 2019 for training/validation
train_df = df[df['Date'] <= '2018-12-31']

# Data after 2018/12/31 for future validation
future_validation_df = df[df['Date'] > '2018-12-31']


In [14]:
train_df

,Date,Open,High,Low,Close,Adj Close,Volume
0,2007-05-03,21.750000,21.950001,20.549999,20.719999,18.365900,8052800
1,2007-05-04,21.250000,21.350000,20.700001,20.980000,18.596355,5437300
2,2007-05-07,21.200001,21.290001,20.200001,20.360001,18.046801,2646300
3,2007-05-08,20.000000,20.000000,19.299999,19.629999,17.399738,4166100
4,2007-05-09,19.750000,20.200001,19.629999,19.790001,17.541559,7541100
...,...,...,...,...,...,...,...
2932,2018-12-24,49.250000,49.450001,47.900002,47.959999,46.274246,5381300
2933,2018-12-26,48.400002,50.459999,47.910000,50.410000,48.638130,7452700
2934,2018-12-27,50.029999,50.500000,48.320000,50.500000,48.724968,7136300
2935,2018-12-28,50.529999,51.099998,49.709999,50.180000,48.416214,5854800


In [15]:
future_validation_df

,Date,Open,High,Low,Close,Adj Close,Volume
2937,2019-01-02,49.250000,50.160000,48.880001,50.090000,48.329384,5270700
2938,2019-01-03,48.509998,48.939999,45.080002,45.610001,44.006847,18694100
2939,2019-01-04,46.369999,48.830002,46.240002,47.790001,46.110226,11037000
2940,2019-01-07,48.000000,48.439999,47.130001,47.849998,46.168114,9375500
2941,2019-01-08,48.480000,48.669998,46.950001,47.470001,45.801468,8231400
...,...,...,...,...,...,...,...
4119,2023-09-13,39.500000,40.099998,39.279999,39.549999,39.549999,11263000
4120,2023-09-14,40.200001,40.279999,38.930000,39.330002,39.330002,13190400
4121,2023-09-15,39.299999,40.110001,39.110001,39.900002,39.900002,9841100
4122,2023-09-18,39.580002,39.830002,38.730000,38.889999,38.889999,8517700


In [16]:
# Save to new CSV files
train_df.to_csv('./Dal/train_dataset_DAL.csv', index=False)
future_validation_df.to_csv('./Dal/future_validation_dataset_DAL.csv', index=False)


In [65]:
import pandas as pd
import numpy as np

# Read the CSV
dataframe = pd.read_csv('./Dal/train_dataset_DAL.csv')
assert 'Date' in dataframe.columns and 'Adj Close' in dataframe.columns, "The CSV should have 'date' and 'close' columns"

# Convert the 'date' column to datetime format
dataframe['Date'] = pd.to_datetime(dataframe['Date'])

# # Filter out the dataset until 2018/12/31 only
# dataframe = dataframe[dataframe['date'] <= '2018-12-31']

# Function to create the dataset
def create_dataset(prices, look_back, label_window):
    X, y, dates = [], [], []
    for i in range(len(prices) - look_back - label_window + 1):
        X.append(prices[i:(i+look_back)])
        dates.append(dataframe['Date'].iloc[i + look_back - 1])  # Store the last date of the look_back window
        
        if prices[i + look_back + label_window - 1] > prices[i + look_back - 1]:
            y.append('Up')
        else:
            y.append('Down')
    return np.array(X), np.array(y), dates

prices = dataframe['Adj Close'].values

look_back = 360
label_window = 20

X, y, date_list = create_dataset(prices, look_back, label_window)

# If you want to save this dataset to a CSV file:
dataset_df = pd.DataFrame({
    'Date': date_list,
    'features': list(X),
    'labels': y
})
dataset_df.to_csv('./Dal/dataset_lookback_360_labelwindow_20.csv', index=False)


In [66]:
df_2 = pd.read_csv("./Dal/dataset_lookback_360_labelwindow_20.csv") 

In [67]:
df_2

,Date,features,labels
0,2008-10-03,[18.3659 18.596355 18.046801 17.399738 17.54...,Up
1,2008-10-06,[18.596355 18.046801 17.399738 17.541559 17.46...,Up
2,2008-10-07,[18.046801 17.399738 17.541559 17.461788 17.10...,Up
3,2008-10-08,[17.399738 17.541559 17.461788 17.107229 16.74...,Up
4,2008-10-09,[17.541559 17.461788 17.107229 16.743814 16.13...,Up
...,...,...,...
2553,2018-11-23,[49.084312 49.186562 49.140083 49.112206 50.04...,Down
2554,2018-11-26,[49.186562 49.140083 49.112206 50.041656 50.06...,Down
2555,2018-11-27,[49.140083 49.112206 50.041656 50.069538 49.94...,Down
2556,2018-11-28,[49.112206 50.041656 50.069538 49.948711 50.24...,Down


In [68]:
df_2['labels'].value_counts()

Up      1444
Down    1114
Name: labels, dtype: int64